## Gather Offensive Rating (ORtg), Defensive Rating (DRtg), and Net Rating (NRtg) for every team for every season 1998-2023

Import necessary packages

In [10]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

Prep URL for 1998-2023 seasons

In [11]:
YEARS = [y for y in range(1998, 2024)]
def make_link (year):
    return f"https://www.basketball-reference.com/leagues/NBA_{year}.html"

Scrape ORtg, DRtg, and NRtg Data for every team from 1998-2023 seasons

In [14]:
data = []

for year in YEARS:
    page = requests.get(make_link(year))
    time.sleep(2)
    soup = BeautifulSoup(page.content, "html.parser")
    table = soup.find("table", {"id": "advanced-team"})
    if table:
        rows = table.find_all("tr")
        
        for row in rows[1:]:
            cells = row.find_all(["th", "td"])
            
            team = cells[1].text.strip()   # 'Team' column
            if team != "Team":
                ortg = cells[10].text.strip()  # 'ORtg' column
                drtg = cells[11].text.strip()  # 'DRtg' column
                nrtg = cells[12].text.strip()  # 'NRtg' column
                
                data.append((year, team, ortg, drtg, nrtg))




Export data to csv

In [15]:
data = pd.DataFrame(data, columns=["Year", "Team", "ORtg", "DRtg", "NRtg"])
data.to_csv("rtg_data.csv", index=False)